In [1]:
from forward import solve_forward
from simulation import Simulation
import numpy as np
import mat73
from net import Region_CNN

2022-01-10 12:53:36.458499: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-10 12:53:36.458665: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
sources = np.load('/media/thanos/Elements/thanos/sim_data/sim_type_1/downsampled_dipoles/sources_20TeD.npy')
eeg = np.load('/media/thanos/Elements/thanos/sim_data/sim_type_1/downsampled_dipoles/eeg_20TeD.npy')
fwd = solve_forward()
eeg_topos = mat73.loadmat('/media/thanos/Elements/thanos/sim_data/sim_type_1/downsampled_dipoles/eeg_20TeD_topos.mat')['eeg_topos']
eeg_topos = eeg_topos.transpose(2, 0, 1)

sim = Simulation(fwd=fwd, source_data=sources, eeg_data=eeg)
region_cnn = Region_CNN(sim=sim, eeg_topographies=eeg_topos)

region_cnn.load_nn('/media/thanos/Elements/thanos/nn_trained/Cnn models/eeg_cnn_1.h5')

Forward problem is solved.
Electrodes: (73, 3)
Dipoles: (5046, 3)
Leadfield: (73, 5046)


2022-01-10 12:55:23.097887: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-01-10 12:55:23.101128: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-01-10 12:55:23.102728: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (thanos): /proc/driver/nvidia/version does not exist
2022-01-10 12:55:23.126978: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-01-10 12:55:23.603813: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 553779200 exceeds 10

Loaded model in Region_CNN :
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 65, 65, 32)        320       
                                                                 
 flatten (Flatten)           (None, 135200)            0         
                                                                 
 dense (Dense)               (None, 1024)              138445824 
                                                                 
 batch_normalization (BatchN  (None, 1024)             4096      
 ormalization)                                                   
                                                                 
 dense_1 (Dense)             (None, 1024)              1049600   
                                                                 
 batch_normalization_1 (Batc  (None, 1024)             4096      
 hNormalization)           

2022-01-10 12:55:30.174156: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 553779200 exceeds 10% of free system memory.


In [9]:
import scipy.io
topos_eval =  scipy.io.loadmat('../../Downloads/eeg_topo_real.mat')['Zi']

topos_eval= np.expand_dims(topos_eval, axis=0)

In [10]:
predicted_sources = region_cnn.predict(eeg=topos_eval).T

In [ ]:
np.mean(predicted_sources)

In [ ]:
np.save('../../Downloads/pred_sources.npy',predicted_sources)